In [1]:
#This script is an attempt to create a 4-element windkessel model for BP simulation

#Author = Joshua Van Tuyl
#Team VitalCentric

#Type = Healthy

import numpy as np
import math
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
from scipy.integrate import solve_ivp



In [2]:
#STEP ONE - Defining Constants
#I'm using integer 1 as a placeholder until I find real values

#Total Peripheral Resistance: Fluid resistance due to periphery 
# mmHg.s.cm^-3
R = 0.79 

#Compliance of veins: Measure of how well the veins stretch during systole and diastole
# cm^3.mmHg^-1.s^2.cm^-3
C = 1.22

#Characteristic impedance: Resistance due to aortic and pulmonary valve
# mmHg.s.cm^-3
r = 0.056

#Inertance: Measure of the blood fluid inertia
# mmHg.s^2.cm^-3
L = 0.0051

In [15]:
#STEP TWO - Defining ODEs
#The 4 element wk model are two coupled linear odes
#Need to truncate into one function



#Also need to define input signal
import numpy as np
import matplotlib.pyplot as plt

def q(t):
    # Period of repetition
    T = 0.8
    
    # Fold time into one period (modulo operation)
    t_mod = np.mod(t, T)
    
    # Initialize array of zeros
    current = np.zeros_like(t)
    
    # Apply waveform within active pulse window
    mask = (t_mod > 0) & (t_mod < 0.3)
    current[mask] = 500 * (np.sin(np.pi * t_mod[mask] / 3))**2
    
    return current

# Example usage
t = np.linspace(0, 3.2, 2000)  # 4 periods
i_values = q(t)

plt.plot(t, i_values)
plt.xlabel('Time (s)')
plt.ylabel('Current i(t)')
plt.title('Periodic Current Pulse (Period = 0.8 s)')
plt.grid(True)
plt.show()

def dSdt(t, S):
    P, q = S
    return [(-1/(R*C))*P + (1/C)*q,
           (-r/L)*q + (r/L)*q]
P_0 = 5
q_0 = 500
S_0 = (P_0, q_0)

In [16]:
t = np.linspace(0, 100, 100)
soln = odeint(dSdt, y0=S_0, t=t, tfirst = True)

In [17]:
P_soln = soln.T[0]

In [18]:
q_soln = []
for t in range(0,100):
    i = q(t)
    q_soln.append(i)

In [23]:
#CHATGPT CODE - NEED TO VERIFY

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Windkessel parameters (example physiological values)
R1 = 0.056    # proximal resistance [mmHg·s/mL]
R2 = 0.79       # peripheral resistance [mmHg·s/mL]
C  = 1.22     # compliance [mL/mmHg]
L  = 0.0051   # inductance [mmHg·s^2/mL]
P_out = 0.0   # outflow/venous pressure [mmHg]

# Define input pressure waveform (e.g., aortic pressure pulse)
def P_ao(t):
    # Simple periodic input: systolic pulse every 1s
    return 100 + 20*np.sin(2*np.pi*t) * (np.sin(np.pi*t)**2) * (t % 1 < 1)

# Differential equations
def windkessel_4element(t, y):
    Q, P = y
    dQdt = (P_ao(t) - R1*Q - P) / L
    dPdt = (Q - (P - P_out)/R2) / C
    return [dQdt, dPdt]

# Simulation time
t_span = (0, 3)        # 3 seconds
t_eval = np.linspace(*t_span, 1000)
y0 = [0, 0]            # initial flow and pressure

# Solve system
sol = solve_ivp(windkessel_4element, t_span, y0, t_eval=t_eval)

# Extract solutions
Q = sol.y[0]
P = sol.y[1]
t = sol.t

# Plot results
plt.figure(figsize=(10,5))
plt.subplot(2,1,1)
plt.plot(t, P, label='Pressure P(t)')
plt.ylabel('Pressure [mmHg]')
plt.legend()
plt.grid(True)

plt.subplot(2,1,2)
plt.plot(t, Q, label='Flow Q(t)', color='orange')
plt.xlabel('Time [s]')
plt.ylabel('Flow [mL/s]')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()